In [5]:
import httpx
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
base_url  = 'https://api.openai.com/v1'
server_url = 'http://localhost:8811'

### **Request** Structure
```json
{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "some.method",
    "params":{...}
}
```

### **Response** Structure
*Success*
```json
{
    "jsonrpc": "2.0",
    "id":1,
    "result": {...}
}
```
*Failure*
```json
{
    "jsonrpc": "2.0",
    "id": 1,
    "error": {
        "code": 123,
        "message": "Something broke"
    }
}
```